In [1]:
import os
import pandas as pd

In [2]:
fastq_path = '/mnt/nupeb/rede-micro/datasets/raw-fastqs/raw-paired-end-fastq'
data_folder_path = '/mnt/nupeb/rede-micro/redemicro-jennefer/data'
sample_ids_file_path = f'{data_folder_path}/vs-sample-ids.tsv'
manifest_file_path = f'{data_folder_path}/manifest.csv'
raw_metadata_file_path = f'{data_folder_path}/metadata-raw.tsv'
metadata_file_path = f'{data_folder_path}/metadata.tsv'
histec_metadata_file_path = f'{data_folder_path}/metadata-his.tsv'
not_histec_metadata_file_path = f'{data_folder_path}/metadata-not-his.tsv'

In [3]:
# Read samples IDs
samples_df = pd.read_csv(sample_ids_file_path, sep='\t', header=None, names=['sample-id', 'vs-id'])
samples_dict = pd.Series(samples_df['vs-id'].values, index=samples_df['sample-id'].values).to_dict()
file_names  = !ls {fastq_path}

# Manifest

In [4]:
header_names = ('sample-id', 'absolute-filepath', 'direction')
header_line = ','.join(header_names)
manifest_lines = [header_line]
for f_name in file_names:
    s_id = int(f_name[:12])
    if s_id in samples_dict.keys():
        full_path = f'{fastq_path}/{f_name}'
        direction = 'forward' if f_name[-11:-10] == '1' else 'reverse'
        line = f'{samples_dict[s_id]},{full_path},{direction}'
        manifest_lines.append(line)

In [5]:
with open(manifest_file_path, 'w') as f:
    f.write('\n'.join(manifest_lines))

# Metadata

In [6]:
metadata_df = pd.read_csv(raw_metadata_file_path, sep='\t', skiprows=[1])
metadata_df['class_meno'] += 1

In [7]:
histerectomia_ids = ['M40', 'M56', 'M76', 'M80', 'M97', 'M100', 'M111', 'M113', 'M124', 'M160']
histerectomia_rows = metadata_df[metadata_df['id_fezes'].isin(histerectomia_ids)].index
metadata_df.iloc[histerectomia_rows, -1] = 0

In [14]:
metadata_df.rename(columns={'id_vaginal': 'sample-id', 'idade': 'age', 'class_meno': 'class'}, inplace=True)
projection_metadata_df = metadata_df[['sample-id', 'age', 'class']]

projection_metadata_df.to_csv(metadata_file_path, sep='\t', index=False)

# Insert new line with data type info for QIMME2
types_line = ['#q2:types', 'numeric', 'categorical']
projection_metadata_df.loc[-1] = types_line
projection_metadata_df.index = projection_metadata_df.index + 1
projection_metadata_df.sort_index(inplace=True)
projection_metadata_df

projection_metadata_df.to_csv(histec_metadata_file_path, sep='\t', index=False)
projection_metadata_df[projection_metadata_df['class']!=0].to_csv(not_histec_metadata_file_path, sep='\t', index=False)

/tmp/ipykernel_144110/3956128100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projection_metadata_df.loc[-1] = types_line
/tmp/ipykernel_144110/3956128100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projection_metadata_df.sort_index(inplace=True)
